In [ ]:
"""
Project Manager Agent for Multi-Agent System
- Manages tasks and assignments
- Coordinates programmer agents
- Provides feedback and improvement suggestions
- Uses small (~1B parameter) LLM for reasoning
"""

import asyncio
import json
import uuid
from typing import Dict, List, Optional, Union, Any, Tuple
from dataclasses import dataclass, field
import logging

from base_agent_framework import (
    AgentRole, ExpertiseArea, MessageType, Task, Message,
    LocalLLMManager, KnowledgeBase, ThoughtMemory, Thought
)

logger = logging.getLogger(__name__)

@dataclass
class ProgrammerMetrics:
    """Metrics tracked for each programmer agent"""
    task_completion_rate: float = 0.0
    code_quality_score: float = 0.0
    knowledge_application: float = 0.0
    learning_progress: Dict[str, float] = field(default_factory=dict)
    recent_performance: List[float] = field(default_factory=list)
    
@dataclass
class ProjectStatus:
    """Status tracker for the overall project"""
    total_tasks: int = 0
    completed_tasks: int = 0
    in_progress_tasks: int = 0
    blocked_tasks: int = 0
    task_completion_rate: float = 0.0
    programmer_performance: Dict[str, ProgrammerMetrics] = field(default_factory=dict)

class ProjectManagerAgent:
    """Project Manager agent implementation using locally deployed small LLMs"""
    
    def __init__(
        self,
        agent_id: str,
        llm_manager: LocalLLMManager,
        knowledge_base: KnowledgeBase,
        message_queue: asyncio.Queue,
    ):
        self.agent_id = agent_id
        self.role = AgentRole.PROJECT_MANAGER
        self.llm_manager = llm_manager
        self.knowledge_base = knowledge_base
        self.message_queue = message_queue
        
        # Internal state
        self.task_list: List[Task] = []
        self.programmer_registry: Dict[str, Dict[str, Any]] = {}
        self.project_status = ProjectStatus()
        self.thought_memory = ThoughtMemory(max_thoughts=1000)
        
        # Initialize task ID counter
        self.next_task_id = 1
        
        logger.info(f"Project Manager Agent {agent_id} initialized")
    
    async def start(self):
        """Start the agent's main processing loop"""
        logger.info(f"Project Manager Agent {self.agent_id} starting")
        while True:
            try:
                message = await self.message_queue.get()
                await self.process_message(message)
                self.message_queue.task_done()
            except Exception as e:
                logger.error(f"Error processing message: {e}")
                import traceback
                logger.error(traceback.format_exc())

    async def process_message(self, message: Message):
        """Process incoming messages based on their type"""
        logger.info(f"Processing message from {message.sender}: {message.message_type}")
        
        if message.recipient != self.agent_id and message.recipient != "all":
            return  # Not for this agent
            
        # Generate thoughts about the message
        context = f"Message from {message.sender} of type {message.message_type}:\n"
        context += json.dumps(message.content, indent=2)
        
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Observation",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
        
        # Process based on message type
        if message.message_type == MessageType.STATUS_UPDATE:
            await self.handle_status_update(message)
        elif message.message_type == MessageType.TECHNICAL_QUERY:
            await self.handle_technical_query(message)
        elif message.message_type == MessageType.REVIEW_REQUEST:
            await self.handle_review_request(message)
        elif message.message_type == MessageType.LEARNING_UPDATE:
            await self.handle_learning_update(message)
        else:
            logger.warning(f"Unknown message type: {message.message_type}")
    
    async def handle_status_update(self, message: Message):
        """Handle status update messages from programmer agents"""
        sender = message.sender
        content = message.content
        
        # Update task status if provided
        if "task_id" in content and "status" in content:
            task_id = content["task_id"]
            new_status = content["status"]
            
            # Find the task and update its status
            for task in self.task_list:
                if task.id == task_id:
                    old_status = task.status
                    task.status = new_status
                    
                    # Generate thought about status change
                    context = f"Task {task_id} changed status from {old_status} to {new_status}"
                    thought = await self.llm_manager.generate_thought(
                        agent_role=self.role,
                        thought_type="Analysis",
                        context=context
                    )
                    
                    if thought:
                        self.thought_memory.add_thought(thought)
                    
                    # Update project status
                    await self.update_project_status()
                    
                    # Provide feedback to the programmer
                    await self.provide_feedback(sender, task)
                    break
        
        # Update programmer metrics if provided
        if "metrics" in content:
            metrics = content["metrics"]
            if sender in self.programmer_registry:
                # Update existing metrics
                self.programmer_registry[sender]["metrics"].update(metrics)
            else:
                # Create new entry
                self.programmer_registry[sender] = {
                    "metrics": metrics,
                    "expertise": content.get("expertise", [])
                }
    
    async def handle_technical_query(self, message: Message):
        """Handle technical queries from programmer agents"""
        sender = message.sender
        content = message.content
        
        # In a real PM agent, we'd consider if this needs to be forwarded to an architect
        # For now, we'll just respond with knowledge base info
        query = content.get("query", "")
        
        # Query the knowledge base
        knowledge_results = await self.knowledge_base.query_knowledge(query)
        
        # Get relevant thoughts
        relevant_thoughts = self.thought_memory.get_relevant_thoughts(query)
        thought_context = "\n".join([
            f"- {t.thought_type}: {t.content} (confidence: {t.confidence})" 
            for t in relevant_thoughts
        ])
        
        # Prepare context for response generation
        context = f"Technical query from {sender}: {query}\n\n"
        context += "Relevant knowledge:\n" + "\n".join(knowledge_results) + "\n\n"
        context += "Relevant past thoughts:\n" + thought_context
        
        # Generate response thought
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Decision",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
            
            # Send response back to programmer
            response_message = Message(
                sender=self.agent_id,
                recipient=sender,
                message_type=MessageType.TECHNICAL_QUERY,
                content={
                    "response": thought.content,
                    "confidence": thought.confidence,
                    "references": knowledge_results[:2]  # Just send top 2 references
                }
            )
            
            await self.send_message(response_message)
    
    async def handle_review_request(self, message: Message):
        """Handle code review requests from programmer agents"""
        sender = message.sender
        content = message.content
        
        code = content.get("code", "")
        task_id = content.get("task_id", "")
        
        # Query knowledge base for relevant patterns
        knowledge_results = await self.knowledge_base.query_knowledge(
            f"code review {content.get('language', '')} {content.get('component', '')}"
        )
        
        # Prepare context for review
        context = f"Code review request from {sender} for task {task_id}:\n\n"
        context += f"```\n{code}\n```\n\n"
        context += "Relevant knowledge:\n" + "\n".join(knowledge_results)
        
        # Generate review thought
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Analysis",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
            
            # Request an additional decision thought based on the analysis
            decision_context = f"Based on code review:\n{thought.content}\n\nWhat feedback should be provided?"
            decision_thought = await self.llm_manager.generate_thought(
                agent_role=self.role,
                thought_type="Decision",
                context=decision_context
            )
            
            if decision_thought:
                self.thought_memory.add_thought(decision_thought)
                
                # Generate a learning opportunity
                learning_context = f"Based on this code review, what learning opportunity can be identified for {sender}?"
                learning_thought = await self.llm_manager.generate_thought(
                    agent_role=self.role,
                    thought_type="Learning",
                    context=learning_context
                )
                
                if learning_thought:
                    self.thought_memory.add_thought(learning_thought)
                
                # Send response back to programmer
                response_message = Message(
                    sender=self.agent_id,
                    recipient=sender,
                    message_type=MessageType.REVIEW_REQUEST,
                    content={
                        "task_id": task_id,
                        "feedback": decision_thought.content,
                        "improvements": learning_thought.content if learning_thought else "",
                        "approved": "approved" in decision_thought.content.lower()
                    }
                )
                
                await self.send_message(response_message)
    
    async def handle_learning_update(self, message: Message):
        """Handle learning updates from programmer agents"""
        sender = message.sender
        content = message.content
        
        learning_area = content.get("area", "")
        progress = content.get("progress", 0.0)
        notes = content.get("notes", "")
        
        # Update programmer metrics
        if sender in self.programmer_registry:
            if "learning_progress" not in self.programmer_registry[sender]:
                self.programmer_registry[sender]["learning_progress"] = {}
                
            self.programmer_registry[sender]["learning_progress"][learning_area] = progress
        
        # Generate thought about learning progress
        context = f"Learning update from {sender} on {learning_area}:\n{notes}\nProgress: {progress}"
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Learning",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
            
        # Query knowledge base for related resources
        knowledge_results = await self.knowledge_base.query_knowledge(learning_area)
        
        # Prepare context for suggestion generation
        suggestion_context = f"Learning update from {sender} on {learning_area}:\n{notes}\n\n"
        suggestion_context += "Relevant knowledge:\n" + "\n".join(knowledge_results)
        
        # Generate suggestion thought
        suggestion_thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Decision",
            context=suggestion_context
        )
        
        if suggestion_thought:
            self.thought_memory.add_thought(suggestion_thought)
            
            # Send learning suggestions
            response_message = Message(
                sender=self.agent_id,
                recipient=sender,
                message_type=MessageType.LEARNING_UPDATE,
                content={
                    "area": learning_area,
                    "suggestions": suggestion_thought.content,
                    "resources": knowledge_results[:2]  # Just top 2 resources
                }
            )
            
            await self.send_message(response_message)
    
    async def create_task(self, title: str, description: str, 
                        required_expertise: List[ExpertiseArea], priority: int = 1) -> Task:
        """Create a new task and add it to the task list"""
        task_id = f"TASK-{self.next_task_id}"
        self.next_task_id += 1
        
        task = Task(
            id=task_id,
            title=title,
            description=description,
            required_expertise=required_expertise,
            priority=priority
        )
        
        self.task_list.append(task)
        self.project_status.total_tasks += 1
        
        # Generate thought about new task
        context = f"Created new task: {title}\n{description}\nRequired expertise: {required_expertise}"
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Decision",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
        
        logger.info(f"Created new task: {task_id} - {title}")
        return task
    
    async def assign_task(self, task: Task) -> bool:
        """Assign a task to the most suitable programmer"""
        # Find programmers with matching expertise
        candidates = []
        for prog_id, prog_info in self.programmer_registry.items():
            expertise_match = any(
                exp in prog_info.get("expertise", []) 
                for exp in task.required_expertise
            )
            
            if expertise_match:
                # Count current assigned tasks
                current_tasks = sum(
                    1 for t in self.task_list 
                    if t.assigned_to == prog_id and t.status != "completed"
                )
                
                candidates.append((prog_id, current_tasks, prog_info))
        
        if not candidates:
            logger.warning(f"No suitable programmers found for task {task.id}")
            return False
            
        # Sort by workload (number of current tasks)
        candidates.sort(key=lambda x: x[1])
        selected_programmer = candidates[0][0]
        
        # Assign the task
        task.assigned_to = selected_programmer
        task.status = "assigned"
        
        # Generate thought about assignment
        context = f"Assigned task {task.id} to {selected_programmer}"
        thought = await self.llm_manager.generate_thought(
            agent_role=self.role,
            thought_type="Decision",
            context=context
        )
        
        if thought:
            self.thought_memory.add_thought(thought)
        
        # Send assignment message
        assignment_message = Message(
            sender=self.agent_id,
            recipient=selected_programmer,
            message_type=MessageType.TASK_ASSIGNMENT,
            content={
                "task_id": task.id,
                "title": task.title,
                "description": task.description,
                "priority": task.priority
            }
        )
        
        await self.send_message(assignment_message)
        logger.info(f"Assigned task {task.id} to {selected_programmer}")
        return True
    
    async def update_project_status(self):
        """Update project status metrics"""
        # Count tasks by status
        completed = sum(1 for task in self.task_list if task.status == "completed")
        in_progress = sum(1 for task in self.task_list if task.status in ["assigned", "in_progress"])
        blocked = sum(1 for task in self.task_list if task.status == "blocked")
        
        # Update status
        self.project_status.completed_tasks = completed
        self.project_status.in_progress_tasks = in_progress
        self.project_status.blocked_tasks = blocked
        
        # Calculate completion rate